# Locais de referência para documentação:

https://ccxt.readthedocs.io/en/latest/manual.html

https://github.com/ccxt/ccxt/issues/8498



# 1 . Carregas as bibliotecas e as chaves para acesso via API

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
!pip3 install ccxt

     |████████████████████████████████| 2.6 MB 9.4 MB/s 
     |████████████████████████████████| 3.6 MB 5.4 MB/s 
     |████████████████████████████████| 271 kB 43.9 MB/s 
     |████████████████████████████████| 1.1 MB 36.1 MB/s 
     |████████████████████████████████| 1.1 MB 36.1 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 291 kB 41.8 MB/s 
     |████████████████████████████████| 144 kB 52.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
#as chaves de acesso estão presentes no arquivo keys.py
from google.colab import files
import keys
from keys import var_apiKey,var_secret

In [ ]:
#var_apiKey = "NY03UwGKcJeGtYYlmeXM61ZtSGQPMdYVMOvx9Jt-"
#var_secret = "MXZGFTLrP3KJzX4kjwx03taloTwh7eRV3E3xs9kX"

In [ ]:
#var_apiKey = "G1JO4qlc05TUYcwI0VEMzcvV60FQ8GHGJTDMMaFU"
#var_secret = "e62u67JQMWc8uQ_Mu9KxwksKjGriVLHpNu_yhqnC"

In [4]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [5]:
# CCXT é a biblioteca que contém as funções de comunicação via API
import asyncio
import ccxt.async_support as ccxt  # noqa: E402

# 2. Cria a instância da corretora FTX

In [6]:
# account poderia ser um vetor com outras contas.
account = {'exchange_id': 'ftx', 'params': {'id': 'FTX', 'apiKey': var_apiKey, 'secret': var_secret}}
exchange_class = getattr(ccxt, account['exchange_id'])
exchange = exchange_class(account['params'])

#3. Funções para carregar a Carteira

In [7]:
async def fetch_my_balance():
        balance = await exchange.fetch_balance()
        await exchange.close()
        carteira = balance
        return(carteira)

In [8]:
# já cria o df e manipula as colunas
def carrega_carteira():
#  carteira = exchange.fetch_balance ()
  carteira = asyncio.get_event_loop().run_until_complete(fetch_my_balance())
  dfcarteira = pd.DataFrame(data = carteira['info']['result']) #, columns=('Moeda','Saldo','Saldo Disponivel','Valor em USD'))
  dfcarteira = dfcarteira[['coin','total','availableWithoutBorrow','usdValue']]
  dfcarteira.columns=('Moeda','Saldo','Saldo Disponivel','Valor em USD')

  dfcarteira.Saldo = dfcarteira.Saldo.astype(float)
  dfcarteira['Saldo Disponivel'] = dfcarteira['Saldo Disponivel'].astype(float)
  dfcarteira['Valor em USD'] = dfcarteira['Valor em USD'].astype(float)

  dfcarteira = dfcarteira[dfcarteira['Saldo'] != 0]

  dfcarteira['Valor Investido'] = int(0)
  dfcarteira['Lucro'] = int(0)
  dfcarteira['Percentual'] = int(0)
  dfcarteira['Valor Compra'] = int(0)
  dfcarteira['Valor Atual'] = int(0)

  return(dfcarteira)


#4. Funções para carregar todas as Trades e filtra as que não viraram ordens

---



In [9]:
# chamada no modo async
async def fetch_my_trades():
        var_since = exchange.parse8601('2021-01-01T00:00:00Z')
        trades = await exchange.fetchMyTrades(since=var_since)
        await exchange.close()
        temp = trades
        return(temp)

In [10]:
# carrega as ordens
def carrega_trades():


  all_trades = asyncio.get_event_loop().run_until_complete(fetch_my_trades())
  df_alltrades = pd.DataFrame(data = all_trades)

  # trata os dicionatios dentro do dicionario, criando novas colunas com essas chaves 
  # e populando-as com os valores

  dict_temp = df_alltrades.loc[0,'info']
  for k in dict_temp.keys():
    df_alltrades.loc[:,'info_'+k] = dict_temp[k]

  dict_temp = df_alltrades.loc[0,'fee']
  for k in dict_temp.keys():
    df_alltrades.loc[:,'fee_'+k] =  dict_temp[k]

  # cria o df das NEGOCIACOES QUE VIRARAM ordens,
  #retirando aquelas ORDERS que nao foram executadas

  df_trades = df_alltrades[~df_alltrades.order.isna()]
  df_trades = df_trades.assign(fator=0)
  df_trades = df_trades.assign(fatorquantidade=0)
  df_trades = df_trades.assign(fatorvalor=0)
  df_trades = df_trades.assign(quantacumulado=0)
  df_trades = df_trades.assign(valormedio=0)
  df_trades = df_trades.assign(valorinvestido=0)

  return(df_trades,df_alltrades)

#5. Funções para cálculos de médias

In [11]:
## Funcao para criar valores que serão usados posteriormente para cálculo das médias
def calcaux(df):
  # separo o df desta moeda em duas partes: compra e venda

  # aqui vou tratar as informaçoes sobre vendas
  dfvenda = df.loc[df['side'] == 'sell']           # filtro somente o que é sell
  dfvenda['fator'] = int(-1)                       # a variácel fator receve -1, que simboliza venda
  dfvenda['fatorvalor'] = dfvenda['fator'] * dfvenda['price'] * dfvenda['amount']  # representa o montande em dollar do vendido
  dfvenda['fatorquantidade'] = dfvenda['fator'] * dfvenda['amount']                # representa a quantidade vendida
  # avgFillPrice => valor = price
  # filledSize   => quantidade = amount

  # aqui vou tratar as informaçoes sobre compras
  dfcompra = df.loc[df['side'] == 'buy'] # filtro somente o que é buy
  dfcompra['fator'] = int(1)
  dfcompra['fatorvalor'] = dfcompra['fator'] * dfcompra['price'] * dfcompra['amount']
  dfcompra['fatorquantidade'] = dfcompra['fator'] * dfcompra['amount']
  
  df = pd.concat([dfcompra, dfvenda])   # junto o df que fora dividido em compre e venda
  df.sort_values(['timestamp','id'], ascending=True,inplace=True)  # ordeno por data
  return(df)

In [12]:
# funcao que calcula a média ponderada dos valor de compra
def calc_mediaponderada(df):

#8 side       (buy ou sell)
#9 price      valor unitário do token
#10 amount    quantidade
#11 cost      valor total = token * quantidade
#12 fee
#33 fator     1 para compra e -1 para venda
#34 fatorquantidade
#35 fatorvalor
#36 quantacumulado
#37 valormedio
#38 valorinvestido

  for x in range(len(df)):
    if df.iloc[x,8] == 'buy':
      df.iloc[x,33] =  1    # fator
    else:
      df.iloc[x,33] = -1    # fator


    df.iloc[x,34] = df.iloc[x,33]*df.iloc[x,10]       #34 fatorquantidade
    df.iloc[x,35] = df.iloc[x,33]*df.iloc[x,11]       #35 fatorvalor
    if x == 0:
      df.iloc[0,36] = df.iloc[0,33]*df.iloc[0,10]      # quantidade acumulada
      df.iloc[0,37] = df.iloc[0,33]*df.iloc[0,9]       # valor medio
    else:
      if df.iloc[x,8] == 'buy':
        df.iloc[x,36] = df.iloc[x-1,36] + df.iloc[x,10]           #36 quantacumulado
        df.iloc[x,37] =(df.iloc[x,9] * df.iloc[x,10]  + df.iloc[x-1,36] * df.iloc[x-1,37] ) / df.iloc[x,36] 
                       # valor atual * quanti atual + media * acumulado / total
        #print(df.iloc[x,37])
      else: # venda
        df.iloc[x,36] = df.iloc[x-1,36] - df.iloc[x,10]          #36 quantacumulado
        df.iloc[x,37] = df.iloc[x-1,37]          #37 valormedio  # quando vende, a media nao muda


    df.iloc[x,38] = df.iloc[x,36] * df.iloc[x,37]               # valor investido baseado no valor medio
    #print(f'vai limpar??: {df.iloc[x,38]}  =  {df.iloc[x,36]}  *  {df.iloc[x,37]}')
    if (df.iloc[x,38] < 0.1):             # quando quantidade em US$ é pequena, representa resíduo e na verdade vendeu tudo        
      df.iloc[x,36] = 0
      df.iloc[x,37] = 0 
      df.iloc[x,38] = 0 
      
  return(df)

In [13]:
## Gera um arquivo com as informações presentes no dataframe
def geraarquivohistorico(df,nome):
#colocar no final as colunas que nao interessam

  cols_at_end = ['info', 'timestamp', 'id', 'order', 'type',
       'takerOrMaker', 'fee', 'info_id',
       'info_market', 'info_future', 'info_baseCurrency', 'info_quoteCurrency',
       'info_type', 'info_side', 'info_price', 'info_size', 'info_orderId',
       'info_time', 'info_tradeId', 'info_feeRate', 'info_fee',
       'info_feeCurrency', 'info_liquidity', 'fee_cost', 'fee_currency',
       'fee_rate', 'fator', 'fatorquantidade', 'fatorvalor']

  #cols_at_end = ['id', 'clientId','status','fator']
  df = df[[c for c in df if c not in cols_at_end] 
        + [c for c in cols_at_end if c in df]]
  #df.head()
  df = df.sort_values('timestamp',ascending=False)
  df.to_csv(nome, header=True,sep=';',decimal=',',index=False)

In [14]:
## acrescenta /USD naquelas que nao sao tipo PERP,  LOCKED nem USD
def nome_completo(acao):
  if 'LOCKED' in acao:
    return(acao)
  elif 'PERP' in acao:
    return(acao)
  elif 'USD' in acao:
    return(acao)
  else:
    return(acao + '/USD')

In [15]:
# coloca no dataframe carteira as informações de média que foram previamente calculadas e presentes no dataframa dados_brutos
# retorna a carteira com os valores de perda e genho
def acrescenta_media(dfdados,dfcarteira):
  valor_medio = 0
  for i in range(len(dfcarteira)):            # loop para cada item na carteira
    acao = dfcarteira.iloc[i]['Moeda']
    acao = nome_completo(acao)           # acrescendo o sufixo no nome do token
    #print(f'2 {acao}')
    try:
      # o valor medio de compra é pego no df trades, pegando o campo valormedio do registro mais recente
      valor_medio = float(dfdados[dfdados['timestamp'] == dfdados[dfdados['symbol']==acao].timestamp.max()].valormedio)
      # para o valor de marcado, é usado novamete a API e pego o valor instantaneo 
      valor_mercado =  exchange.markets[acao]['info']['price'] 
      flag = 1
    except:

      valor_medio = 0
      valor_mercado = 0
      flag = 0

    dfcarteira.iloc[i,4] = round(valor_medio  * dfcarteira.iloc[i]['Saldo'],2)           # 4 é o montante investido
    dfcarteira.iloc[i,5] = round(dfcarteira.iloc[i,3] - dfcarteira.iloc[i,4],2)               # 5 é o lucro, sendo 3 o montante corrente
    if flag:
      if (dfcarteira.iloc[i,4] > dfcarteira.iloc[i,3]):
      # deu lucro
        dfcarteira.iloc[i,6] = f'{(dfcarteira.iloc[i,3]/dfcarteira.iloc[i,4] - 1 ) * 100:.2f} %'
      else:
      # deu prejuizo
        dfcarteira.iloc[i,6] = f'{-(1 - dfcarteira.iloc[i,3]/dfcarteira.iloc[i,4]) * 100:.2f} %'

      dfcarteira.iloc[i,7] = valor_medio
      dfcarteira.iloc[i,8] = valor_mercado
    #print(valor_mercado)
    #print(acao,dfcarteira.iloc[i]['Valor Investido'])
  #print(saldo['Valor Investido'])
  return(dfcarteira)

#6.  Funções para carregar as ordens abertas (FecthOpenOrders) para ordens NORMAIS


In [16]:
# chamada no modo async
async def fetch_my_openorders():
  exchange.options['fetchOpenOrders']['method'] = None
  ordens = await exchange.fetchOpenOrders()
  await exchange.close()
  temp = ordens
  return(temp)

In [17]:
def carrega_ordens():
  df_openorderssummary = pd.DataFrame()

  open_orders = asyncio.get_event_loop().run_until_complete(fetch_my_openorders())

  if len(open_orders) > 0 :
    df_openorders = pd.DataFrame(data = open_orders)
    dict_temp = df_openorders.loc[0,'info']
  
    # trata os dicionatios dentro do dicionario, criando novas colunas com essas chaves e populando-as com os valores
    for k in dict_temp.keys():
      df_openorders.loc[:,'info_'+k] = dict_temp[k]

    df_openorderssummary = pd.DataFrame()
    df_openorderssummary[['datetime','symbol','side','price','amount','cost']] = df_openorders[['datetime','symbol','side','price','amount','cost']]
    df_openorderssummary['tipo'] = 'normal'
    df_openorderssummary.cost = df_openorderssummary.price * df_openorderssummary.amount

  return(df_openorderssummary)

#7.  Funções para carregar as ordens abertas (fetchOpenOrders {PARAM}) para ordens GATILHO

In [18]:
# chamada no modo async
async def fetch_my_trigerorders():
  exchange.options['fetchOpenOrders']['method'] = 'privateGetConditionalOrders'
  ordens = await exchange.fetchOpenOrders()
  await exchange.close()
  temp = ordens
  return(temp)

In [19]:
# carrega as ordens tipo gatilho. É a mesma função que carrega as ordens tradicionais, mas com um parametro especial
def carrega_ordensgatilho():
  df_triggerorderssummary = pd.DataFrame()
  if exchange.has['fetchOpenOrders']:
    trigger_orders = asyncio.get_event_loop().run_until_complete(fetch_my_trigerorders())

  if len(trigger_orders) > 0 :
    ## JA TRANSFORMA LOGO EM DATAFRAME
    df_triggerorders = pd.DataFrame(data = trigger_orders)

    # trata os dicionatios dentro do dicionario, criando novas colunas com essas chaves e populando-as com os valores
    dict_temp = df_triggerorders.loc[0,'info']
    for k in dict_temp.keys():
      df_triggerorders.loc[:,'info_'+k] = dict_temp[k]


    df_triggerorderssummary[['datetime','symbol','side','price','amount','cost']] = df_triggerorders[['datetime','symbol','side','price','amount','cost']]
    df_triggerorderssummary['tipo'] = 'gatilho'
    df_triggerorderssummary.cost = df_triggerorderssummary.price * df_triggerorderssummary.amount
  return(df_triggerorderssummary)

#8. MODULO PRINCIPAL

## 8.1 Primeiro carrega todas as cotações

In [20]:
# chamada no modo async
async def load_my_markets():
        trades = await exchange.load_markets(True)    # TRUE FORÇA A ATUALIZAR COM UMA NOVA CONSULTA
        await exchange.close()
        temp = trades
        return(temp)

##8.2 Inicia as variáveis (dataframes) e o processamento

In [21]:
if __name__ == '__main__':
  df_dadosbrutos = pd.DataFrame()
  df_orders =      pd.DataFrame()
  df_carteira =    pd.DataFrame()
  df_trades =      pd.DataFrame()
  df_alltrades =   pd.DataFrame()

  markets = asyncio.get_event_loop().run_until_complete(load_my_markets())  # carrega todas as cotacoes correntes

  df_carteira = carrega_carteira()
  df_trades,df_alltrades = carrega_trades()

  #sao todas as acoes que eu ja tive alguma ordem executada
  listaacoes = df_trades[df_trades['amount'].notnull()].symbol
  listaacoes = list(set(listaacoes))  # remove as duplicadas

  # LOOP PARA CADA ACAO PRESENTE NA CARTEIRA
  for acao in listaacoes:

    #filtrar o df com a acao do loop
    dftemp = df_trades[df_trades['symbol'] == acao]

    #acrescenta as colunas auxiliares
    dftemp = calcaux(dftemp)
 
    #acrescenta as medias
    dftemp = calc_mediaponderada(dftemp) # nova versao. Era #dftemp = calcmedia(dftemp)

    #concatena resultado de cada acao em dadosbrutos
    df_dadosbrutos = pd.concat([df_dadosbrutos, dftemp])
  
    # termino com as médias acrescentadas no df dados brutos, onde estao todas as trades

  #colocar as informacoes em um arquivo
  geraarquivohistorico(df_dadosbrutos,nome='ordens.csv')


  # baseado nas médias presentes em dadosbrutos,
  # calcula os ganhos e perdas e coloca no dataframe carteira
  df_carteira = acrescenta_media(df_dadosbrutos,df_carteira)

  # Vou consultar as ordens em aberto e no final guardar em um arquivo

  df_openorderssummary    = carrega_ordens()
  df_triggerorderssummary = carrega_ordensgatilho()

  if len(df_openorderssummary) > 0:       # CASO TENHA ALGUMA ORDEM ABERTA
    df_orders = df_orders.append(df_openorderssummary)
    if len(df_triggerorderssummary) > 0:  # CASO TENHA ALGUMA ORDEM TRIGGER ABERTA
      df_orders = df_orders.append(df_triggerorderssummary)

    df_orders.sort_values(['symbol'],inplace=True, ascending=False)
    df_orders.to_csv("openorders.csv", header=True,sep=';',decimal=',',index=False)

  df_carteira.sort_values(['Lucro'],inplace=True, ascending=False)
  df_carteira.to_csv("carteira.csv", header=True,sep=';',decimal=',',index=False)

  df_alltrades.sort_values(['timestamp'],inplace=True, ascending=False)
  df_alltrades.to_csv("alltrades.csv", header=True,sep=';',decimal=',',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

#9. Relatórios em tela

##9.1 Lista a Carteria ordenada pela coluna 'Lucro', acrescentando o saldo no cabeçalho.

O campo "Valor Compra" possui o valor medio de compra da moeda, informação está que é usada para calcular o lucro ou prejuízo que é declarado no IR

In [22]:
#markets.keys()
#markets['BTC/USD'].keys()
markets['BTC/USD']['info']['price']

#markets['SRM/USD'].keys()
#markets['SRM/USD']['info']['price']


'39782.0'

In [27]:
temp = 'Valor (' + str(int(df_carteira['Valor em USD'].sum())) + ' USD)'
df_carteira.rename(columns={'Valor em USD' : temp},  inplace = True)
df_carteira

,Moeda,Saldo,Saldo Disponivel,Valor (3824 USD),Valor Investido,Lucro,Percentual,Valor Compra,Valor Atual
4,USD,1.319004e+00,1.319004e+00,1.319004,0.00,1.32,0,0.000000,0
0,SRM_LOCKED,3.654046e-01,3.654046e-01,0.801698,0.00,0.80,0,0.000000,0
7,FTT,1.398614e+00,0.000000e+00,57.390727,84.84,-27.45,-32.35 %,60.658070,41.074
24,SHIB,2.499550e+06,2.499550e+06,58.964385,104.88,-45.92,-43.78 %,0.000042,0.0000236
19,BNB,7.896979e-01,9.697940e-03,311.072068,454.60,-143.53,-31.57 %,575.664740,394.144
5,BTC,4.027748e-02,1.587748e-02,1601.005934,1781.31,-180.30,-10.12 %,44226.053122,39782.0
37,ETH,1.079811e-01,9.811400e-04,317.177858,499.41,-182.23,-36.49 %,4625.000000,2937.2
12,CRO,6.898758e+02,9.875800e+00,271.446307,496.71,-225.26,-45.35 %,0.720000,0.392975
13,GALA,5.598992e+02,9.899200e+00,98.418034,341.93,-243.51,-71.22 %,0.610702,0.1759
25,GRT,3.629347e+02,9.346600e-01,144.242098,399.23,-254.99,-63.87 %,1.100000,0.3981


##9.2 Lista as Ordens abertas, indicando o tipo (normar ou gatinho)

In [28]:
df_orders.head(100)

,datetime,symbol,side,price,amount,cost,tipo
0,2021-12-03T01:27:44.770Z,SHIB/USD,sell,0.00005,2.500000e+06,126.00,gatilho
6,2021-12-26T20:09:54.348Z,POLIS/USD,sell,11.30000,3.890000e+01,439.57,normal
4,2021-12-16T01:23:00.806Z,GRT/USD,sell,1.20000,3.620000e+02,434.40,normal
2,2021-12-07T23:55:59.292Z,GALA/USD,sell,0.67000,5.500000e+02,368.50,normal
5,2021-12-25T23:29:10.816Z,ETH/USD,sell,5000.00000,1.070000e-01,535.00,normal
1,2021-12-07T23:37:37.205Z,ENJ/USD,sell,3.99000,1.100000e+02,438.90,normal
3,2021-12-16T01:20:56.708Z,CRO/USD,sell,0.79000,6.800000e+02,537.20,normal
7,2022-03-29T11:34:33.540Z,BTC/USD,sell,55000.00000,9.900000e-03,544.50,normal
8,2022-04-11T19:53:49.680Z,BTC/USD,sell,45000.00000,1.450000e-02,652.50,normal
0,2021-12-06T23:36:01.088Z,BNB/USD,sell,661.50000,7.800000e-01,515.97,normal


##9.3 Lista as informacões mais relevantes de transações sobre uma determinada moeda. Nesse exemplo BTC/USD

In [29]:
df_dadosbrutos[df_dadosbrutos.symbol == 'SRM/USD'][['datetime','side','quantacumulado','price','amount','cost']].sort_values('datetime',ascending=False)


,datetime,side,quantacumulado,price,amount,cost
64,2021-10-05T16:46:26.617Z,sell,119.0,8.222,1.0,8.222
53,2021-10-01T20:21:14.875Z,buy,120.0,8.200,4.0,32.800
52,2021-10-01T20:21:14.686Z,buy,116.0,8.200,56.0,459.200
51,2021-10-01T19:23:10.227Z,buy,60.0,8.275,60.0,496.500
